In [ ]:
%load_ext autoreload

In [ ]:
from scipy.ndimage import affine_transform
import tifffile as tiff
import os
import numpy as np
import copy
from pathlib import Path
import sys
import matplotlib.pyplot as plt

PIPELINE_ROOT =  Path().resolve().parent.parent
sys.path.append(PIPELINE_ROOT.as_posix())

In [ ]:
from library.image_manipulation.pipeline_process import Pipeline
from library.image_manipulation.elastix_manager import create_downsampled_transforms
from library.utilities.utilities_registration import (
    parameters_to_rigid_transform,
    rigid_transform_to_parmeters,
)
#from settings import host, password, user, schema
channel = 1
downsample = True
debug = False
rescan_number = 0
task = 'align'

In [ ]:
class ManualAligner(Pipeline):
    
    def __init__(self, animal, section):
        """"
        below midfile
        increase xshift of section pushes preceding images to the right
        decrease xshift of section pushes preceding images to the left
        increase yshift of moving pushes preceding images down
        decrease yshift of moving pushes preceding images up
        increase rotation of moving rotates preceding images clockwise
        decrease rotation of moving rotates preceding images counterclockwise
        """
        super().__init__(animal, rescan_number, channel, downsample, 
                         task, debug)
        self.sections = self.sqlController.get_sections(self.animal, self.channel, rescan_number)
        self.midpoint = len(self.sections) // 2
        self.moving_index = section - 1
        self.fixed_index = section
        INPUT = os.path.join(self.fileLocationManager.prep, 'C1', 'thumbnail_aligned')
        self.moving_file = os.path.join(INPUT, f'{str(self.moving_index).zfill(3)}.tif')
        self.fixed_file = os.path.join(INPUT, f'{str(self.fixed_index).zfill(3)}.tif')
        self.moving = tiff.imread(self.moving_file)
        self.fixed = tiff.imread(self.fixed_file)
        self.transformations = self.get_transformations()
        self.center = self.get_rotation_center()
        
    def transform_image(self, img, T):
        matrix = T[:2,:2]
        offset = T[:2,2]
        offset = np.flip(offset)
        img = affine_transform(img, matrix.T, offset)
        return img
    
    def evalulate_transformation(self, fixed, moving, Tf, Tm):
        fixed = self.transform_image(fixed, Tf)
        moving = self.transform_image(moving, Tm)
        self.compare_two_image(fixed, moving)
        
    def image_side_by_side(self, fixed, moving, Tf=None, Tm=None):
        if Tm is not None:
            fixed = self.transform_image(fixed, Tf)
            moving = self.transform_image(moving, Tm)
        fig, ax = plt.subplots(1,2)
        fig.set_figheight(15)
        fig.set_figwidth(15)
        ax[0].grid()
        ax[1].grid()
        ax[0].title.set_text(f'fixed {self.fixed_index}')
        ax[1].title.set_text(f'moving {self.moving_index}')
        ax[0].imshow(fixed, cmap="gray")
        ax[1].imshow(moving, cmap="gray")
        
    def compare_two_image(self, img1, img2):
        img1,img2 = img1.astype(float),img2.astype(float)
        plt.figure(figsize=[20,20])
        plt.imshow(img1,cmap='Blues')
        plt.imshow(img2,alpha=0.5,cmap='Greys')
         
    def get_transformations(self):
        transformations = {}
        center = self.get_rotation_center()
        for i in range(1, len(self.sections)):
            rotation, xshift, yshift = self.load_elastix_transformation(self.animal, i)
            T = parameters_to_rigid_transform(rotation, xshift, yshift, center)
            transformations[i] = T
        return transformations

    def get_transformation_of_section_to_midpoint(self, section, transformations):
        if section == self.midpoint:
            transformation = np.eye(3)
        elif section < self.midpoint:
            T_composed = np.eye(3)
            for i in range(self.midpoint, section, -1):
                T_composed = np.dot(np.linalg.inv(transformations[i]), T_composed)
                # print(f'midpoint={self.midpoint}, i={i}, section={section}')
            transformation = T_composed
        else:
            # original
            T_composed = np.eye(3)
            for i in range(self.midpoint + 1, section + 1):
                #print(f'midpoint={self.midpoint}, i={i}, section={section}')
                T_composed = np.dot(transformations[i], T_composed)
            transformation = T_composed
            
        
        return transformation
    
    def get_transformation_of_section(self, section):
        transformation = self.get_transformation_of_section_to_midpoint(
            section, self.transformations)
        return transformation

    def get_modified_transformation_of_section(self, fixed_index, rotation, xshift, yshift):
        T = parameters_to_rigid_transform(rotation, xshift, yshift, self.center)
        transformations = copy.copy(self.transformations)
        transformations[fixed_index] = T
        transformation = self.get_transformation_of_section_to_midpoint(
            fixed_index - 1, transformations)
        return transformation
    
    def compare_untransformed(self):
        self.compare_two_image(self.moving,self.fixed)
        
    def evaluate_current_transformation(self):
        Tf = self.get_transformation_of_section(self.fixed_index)
        Tm = self.get_transformation_of_section(self.moving_index)
        self.evalulate_transformation(self.fixed, self.moving, Tf, Tm)
        plt.title(self.moving_index);

In [ ]:
prep_id = 'DK78'
section = 180 # section = fixed_index
aligner = ManualAligner(prep_id, section)
print(f'fixed index={aligner.fixed_index}')
print(f'moving index={aligner.moving_index}')

In [ ]:
#aligner.evaluate_current_transformation()
aligner.compare_two_image(aligner.fixed, aligner.moving)

In [ ]:
aligner.image_side_by_side(aligner.fixed, aligner.moving)

In [ ]:
rotation, xshift, yshift = aligner.load_elastix_transformation(aligner.animal, aligner.fixed_index)
print(f'fixed index = {aligner.fixed_index}')
print(f'rotation of fixed = {rotation}')
print(f'xshift of fixed = {xshift}')
print(f'yshift of fixed = {yshift}')

- preceding image = moving image
- increase xshift of fixed pushes preceding images to the right
- decrease xshift of fixed pushes preceding images to the left
- increase yshift of fixed pushes preceding images down
- decrease yshift of fixed pushes preceding images up
- increase rotation of fixed rotates preceding images clockwise
- decrease rotation of fixed rotates preceding images counterclockwise


In [ ]:
## moving = blue, fixed = grey, previous image

#rotation = 0.3
#xshift = -300
#yshift = -20
T1 = aligner.get_transformation_of_section(aligner.moving_index)
T2 = aligner.get_modified_transformation_of_section(aligner.fixed_index, rotation, xshift, yshift)
aligner.evalulate_transformation(aligner.fixed, aligner.moving, T1, T2)
title = f'Moving blue section {aligner.moving_index} on {prep_id}'
title += f'\nFixed section={aligner.fixed_index} with rotation={rotation}, xshift={xshift}, yshift={yshift}'
plt.title(title);

In [ ]:
#print(f'section={section}')
#aligner.image_side_by_side(aligner.fixed, aligner.moving, Tf, Tm)

In [ ]:
sql = f'UPDATE elastix_transformation SET rotation={rotation}, xshift={xshift}, yshift={yshift} WHERE section=\'{aligner.fixed_index}\' and FK_prep_id=\'{prep_id}\';'
print(sql)